# OMOP to HN Stage

In [ ]:
import mysql.connector
import pandas as pd
import psycopg2
import random
import numpy as np
from datetime import datetime, timedelta

In [ ]:
config = {
    'user': 'user', 
    'password': 'password',
    'host': '127.0.0.1',
    'database': 'idea4rc_dm',
    'raise_on_warnings': True
}

conn2 = mysql.connector.connect(**config)
curIDEA = conn2.cursor()

In [ ]:
conn = psycopg2.connect(
    dbname="omopdb",
    user="postgres",
    password="mysecretpassword",
    host="localhost",
    port="5432"
)

curOMOP = conn.cursor()

In [ ]:
ct_concepts = [1635299, 1634269, 1633589, 1635857, 1635227, 1633737, 1635656, 1633815, 
 1635794, 1634381, 1635664, 1633883, 1633747, 1634029, 1634651, 1633877, 
 1633324, 1635556, 1635522, 1635530, 1634973, 1634247, 1634522, 1634963, 
 1634624, 1634120, 1634854]

cn_concepts = [1635104, 1633679, 1634797, 1633315, 1633942, 1634070, 1635697, 1634139, 
 1633651, 1635470, 1635634, 1633763, 1634143, 1635739, 1633788, 1633433, 1635677, 1633323,
 1634678, 1634727, 1633271, 1635605, 1634037, 1633854, 1633434, 1635004, 1635496, 1635283,
 1635084, 1635828]

cm_concepts=[1634194, 1633468, 1634757, 1634829, 1633276, 1633974]

pt_concepts=[1634270, 1635402, 1634986, 1635660, 1633798, 1634720, 1633279, 1634675, 1634635, 1633445,
 1635422, 1635070, 1634792, 1634491, 1633307, 1635670, 1634658, 1634386, 1635311, 1635341, 1635396, 1634101,
 1633723, 1634894, 1633900, 1633699, 1633658]

pn_concepts=[1635823, 1634505, 1634117, 1634212, 1633726, 1635560, 1634562, 1634245, 1633659, 1634541, 1633569,
 1633336, 1633273, 1635717, 1635871, 1634773, 1633607, 1634645, 1635113, 1634601, 1634383, 1634504, 1633668,
 1635307, 1634271, 1634397, 1635545, 1633500, 1634847, 1634770]

pm_concepts=[1635345, 1635536, 1634606, 633469, 1635336, 634891]

radiological_extranodal_concepts=[36769946, 36770618]

staging_concepts=[1635842, 1633828, 1635824, 1633905, 1634457, 1635758, 1634718, 1635182, 1635217, 1635848, 1635125,
 1634596, 1634307, 1634766, 1635029, 1635535, 1634451, 1634810, 1633922, 1635757, 1635708, 1633270, 1634614, 1635006,
 1634741, 1635511, 1634787, 1635797, 1635800, 1634799, 1633751, 1634619, 1635386, 1633499, 1634947, 1634705, 1634208,
 1635230, 1633697, 1634731, 1635745, 1634005, 1635370, 1634472, 1634487, 1635893, 1634492, 1634551]

queryGetEpisodesCancer="""
    SELECT episode_id FROM episode WHERE episode_type_concept_id=32879;
"""
curOMOP.execute(queryGetEpisodesCancer)
episodes = curOMOP.fetchall()
queryGetMeasurements="""
    SELECT measurement_concept_id, value_as_concept_id, measurement_date,patient_id FROM measurement WHERE measurement_event_id=%s AND meas_event_field_concept_id=798885;
"""

getIDEAEpisodeEvent="""
    SELECT id FROM idea4rc_dm.episode_event WHERE date_of_episode=%s AND cancer_episode=%s;
"""
sqlGetEpisode = """
    SELECT c.id 
    FROM episode c
    WHERE c.person_id = %s
    AND c.episode_start_date = %s
    LIMIT 1
"""
sqlGetDateEpisode = "SELECT episode_start_date FROM episode WHERE episode_id=%s"
getExistshnstage="""
    SELECT * FROM idea4rc_dm.hn_stage WHERE episode_event=%s;
"""
for episode in episodes:
    idEpisode=episode[0]
    curOMOP.execute(queryGetMeasurements, (idEpisode))
    measurements = curOMOP.fetchall()
    for measurement in measurements:
        concept=measurement[0]
        value=measurement[1]
        column=None
        if concept in ct_concepts:
            column="CT"
        elif concept in cn_concepts:
            column="CN"
        elif concept in cm_concepts:
            column="CM"
        elif concept in pt_concepts:
            column="PT"
        elif concept in pn_concepts:
            column="PN"
        elif concept in pm_concepts:
            column="PM"
        elif concept in radiological_extranodal_concepts:
            column="Radiological Extranodal"
        elif concept in staging_concepts:
            column="Staging"
        elif concept == 35225724:
            column="soft_tissue"
        elif concept == 36769243:
            column="distant_lymph_node"
        elif concept == 36770283:
            column="lung"
        elif concept == 36769301:
            column="bone"
        elif concept == 36770544:
            column="liver"
        elif concept == 35226258:
            column="pleura"
        elif concept == 35226253:
            column="peritoneum"
        elif concept == 36768862:
            column="brain"
        elif concept == 36769180:
            column="other_viscera"
        elif concept == 4129922:
            column="unknown"
        curOMOP.execute(sqlGetDateEpisode, (idEpisode))
        dateEpisode = curOMOP.fetchone()
        curIDEA.execute(sqlGetEpisode, (measurement[3], dateEpisode[0]))
        idEpisodeIDEA = curIDEA.fetchone()
        curIDEA.execute(getIDEAEpisodeEvent, (dateEpisode[0], idEpisodeIDEA[0]))
        epEvent=curIDEA.fetcOne()

        if column is not None:
            curIDEA.execute(getExistshnstage, (epEvent[0]))
            if curIDEA.fetchone() is None:
                insert = "INSERT INTO idea4rc_dm.hn_stage (episode_event, {column}) VALUES (%s, %s)"
                curIDEA.execute(insert, (epEvent[0], value))
                conn2.commit()
            else:
                update = "UPDATE idea4rc_dm.hn_stage SET {column}=%s WHERE episode_event=%s"
                curIDEA.execute(update, (value, epEvent[0]))
                conn2.commit()
